# ライブラリ

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# import japanize_matplotlib
import seaborn as sns


from function import create_data

# データ読みこみ

In [2]:
category_names_csv = '../original_data/category_names.csv'
item_categories_csv = '../original_data/item_categories.csv'
sales_history1_csv = '../original_data/sales_history1.csv'
sales_history2_csv = '../original_data/sales_history2.csv'

# DataFrame作成

## Category_name作成

In [3]:
df_category_name = pd.read_csv(category_names_csv, encoding='shift_jis')

- 『商品カテゴリ名』=> 『商品カテゴリ』『商品名』に変換

In [4]:
df_category_name = create_data.split_column_by_delimiter(df_category_name, '商品カテゴリ名', '-', ['商品カテゴリ', '商品名'])

- 商品カテゴリでグループ分け

In [5]:
# 商品カテゴリ列が同じものの中で、商品名列のユニーク値を確認する
result = create_data.check_unique_values(df_category_name, '商品カテゴリ', '商品名')
print(result)

商品カテゴリ
PCゲーム                                                  [通常版]
PCソフト                                                [PCソフト]
ギフト        [小型ゲーム, カード、パズル等, フィギュア, ぬいぐるみ, ボードゲーム, ガジェット類...
ゲーム                         [ゲーム機C, ゲーム機B, ゲーム機A, 付属品(購入特典)]
ゲーム周辺機器                                       [ゲーム機B, ゲーム機A]
映画                              [DVD, ブルーレイ, ブルーレイ3D, セット商品]
本                        [オーディオブック_タイプA, 教育用, オーディオブック_タイプB]
音楽                                           [その他, CD, レコード]
Name: 商品名, dtype: object


## Item_categories作成

In [6]:
item_categories = pd.read_csv(item_categories_csv, encoding='shift_jis')

## Sales_history作成

In [7]:
df_sales_history1 = pd.read_csv(sales_history1_csv, encoding='shift_jis')
df_sales_history2 = pd.read_csv(sales_history2_csv, encoding='shift_jis')

- 合成

In [8]:
# カラム名は等しいので、縦に結合+index初期化
df_sales_history = pd.concat([df_sales_history1, df_sales_history2], ignore_index=True)
df_sales_history.sort_values(by='日付', inplace=True)
df_sales_history.reset_index(drop=True, inplace=True)

df_sales_history['売上'] = df_sales_history['商品価格'] * df_sales_history['売上個数']

- 出力

In [9]:
df_sales_history.to_csv('../processing_data/sales_history.csv', index=False, encoding='shift_jis')

# データ結合

category_names + item_categories + sales_history<br>
主キーは、商品カテゴリ

- category_names + item_categories

In [10]:
# '商品カテゴリID'でマージする
df_temp = pd.merge(item_categories, df_category_name, on='商品カテゴリID', how='inner')

- + sales_history

In [11]:
merged_df = pd.merge(df_sales_history, df_temp, on='商品ID', how='inner')

- ソート

In [12]:
merged_df.sort_values(by='日付', inplace=True)
merged_df.reset_index(drop=True, inplace=True)

- 出力

In [13]:
merged_df.to_csv('../processing_data/merged.csv', encoding='shift_jis', index=False)